## Intro to data engineering example with data from mariadb and csv file

### Load important packages and open a conection to the database

In [54]:
import pandas as pd
from datetime import datetime
import time
import os
from apscheduler.schedulers.background import BackgroundScheduler
import mariadb
import sys

# Connect to MariaDB Platform
#try:
#    conn = mariadb.connect(
#        user="root",
#        password="",
#        host="127.0.0.1",
#        port=3306,
#        database="test"

#    )
#except mariadb.Error as e:
#    print(f"Error connecting to MariaDB Platform: {e}")
#    sys.exit(1)



### Code in charge of the data wrangling operations: load, cleaning, join, aggregate using pandas

In [55]:


print('Data wrangling, the starting time is: %s' % datetime.now())

# Get Cursor
#db_cursor = conn.cursor()
#db_cursor.execute(
#   "SELECT DepartamentName,DepartmentID FROM department")
#table_rows = db_cursor.fetchall()

#df_depart = pd.DataFrame(table_rows, columns = ["DepartamentName", "DepartmentID"])

df_depart = pd.read_csv('dpt.txt', encoding="ISO-8859-1",
                            sep=',')
df_workers = pd.read_csv('wk.txt', encoding="ISO-8859-1",
                             sep=',')
display(df_workers)
display(df_depart)



Data wrangling, the starting time is: 2020-09-23 10:07:06.778759


,Name,DepartmentID,Budget
0,Bob,31,27
1,Marcel,33,46
2,Lisa,33,32
3,Thomas,33,25
4,Tim,34,39
5,Josh,34,32
6,Raphael,36,50
7,Dan,36,120


,DepartamentName,DepartmentID
0,Sales,31
1,Marketing,33
2,HumanResources,34
3,DataScience,35


## Filtering

In [56]:
df_workers.filter(df_workers["Budget"] > 30)
print("After filtering by budget")
display(df_workers)


After filtering by budget


,Name,DepartmentID,Budget
0,Bob,31,27
1,Marcel,33,46
2,Lisa,33,32
3,Thomas,33,25
4,Tim,34,39
5,Josh,34,32
6,Raphael,36,50
7,Dan,36,120


## Aggregating

In [57]:
df_aggregated = df_workers.agg({'Budget': ['sum', 'min']})
print("After aggregating budget")
display(df_aggregated)


After aggregating budget


,Budget
sum,371
min,25


## Inner Join

In [58]:

df_inner = pd.merge(df_depart, df_workers, how='inner', on="DepartmentID")
    
print("After Inner join")    
display(df_inner)
    

After Inner join


,DepartamentName,DepartmentID,Name,Budget
0,Sales,31,Bob,27
1,Marketing,33,Marcel,46
2,Marketing,33,Lisa,32
3,Marketing,33,Thomas,25
4,HumanResources,34,Tim,39
5,HumanResources,34,Josh,32


## Outer Join

In [59]:
df_outer = pd.merge(df_depart, df_workers, how='outer', on="DepartmentID")
print("After Outer join")    
display(df_outer)


After Outer join


,DepartamentName,DepartmentID,Name,Budget
0,Sales,31,Bob,27.0
1,Marketing,33,Marcel,46.0
2,Marketing,33,Lisa,32.0
3,Marketing,33,Thomas,25.0
4,HumanResources,34,Tim,39.0
5,HumanResources,34,Josh,32.0
6,DataScience,35,NaN,NaN
7,NaN,36,Raphael,50.0
8,NaN,36,Dan,120.0


## Left Join

In [60]:
df_left = pd.merge(df_depart, df_workers, how='left', on="DepartmentID")
print("After Left join")        
display(df_left)


After Left join


,DepartamentName,DepartmentID,Name,Budget
0,Sales,31,Bob,27.0
1,Marketing,33,Marcel,46.0
2,Marketing,33,Lisa,32.0
3,Marketing,33,Thomas,25.0
4,HumanResources,34,Tim,39.0
5,HumanResources,34,Josh,32.0
6,DataScience,35,NaN,NaN


## Right Join

In [61]:
df_right = pd.merge(df_depart, df_workers, how='right', on="DepartmentID")
print("After right join")        
display(df_right)

After right join


,DepartamentName,DepartmentID,Name,Budget
0,Sales,31,Bob,27
1,Marketing,33,Marcel,46
2,Marketing,33,Lisa,32
3,Marketing,33,Thomas,25
4,HumanResources,34,Tim,39
5,HumanResources,34,Josh,32
6,NaN,36,Raphael,50
7,NaN,36,Dan,120


## Grouping

In [62]:
    grouped = df_workers.groupby('DepartmentID')
    display(grouped.get_group(33))
    count = len(grouped.get_group(33))
    print("total number of workers that received budget in department 33 is: "+ str(count))
 

,Name,DepartmentID,Budget
1,Marcel,33,46
2,Lisa,33,32
3,Thomas,33,25


total number of workers that received budget in department 33 is: 3


## Concatenating

In [63]:
one = pd.DataFrame({
         'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Luis'],
         'DepartmentID':['33','26','4','11','34']},
         index=[1,2,3,4,5])
two = pd.DataFrame({
         'Name': ['Martin', 'Brian', 'Daniel', 'Ron', 'Betty'],
         'DepartmentID':['4','32','6','33','16']},
         index=[1,2,3,4,5])
display(pd.concat([one,two]))


,Name,DepartmentID
1,Alex,33
2,Amy,26
3,Allen,4
4,Alice,11
5,Luis,34
1,Martin,4
2,Brian,32
3,Daniel,6
4,Ron,33
5,Betty,16


### Which operations do we have to perform if we want to know:
### 1) Which department did not receive budget?
### 2) What is a good approach to fill null values in the budget column after a join?
### 3) Total budget received by each department?
### 4) Department were more workers received budget?

## With scheduler

In [64]:

def move_data():
    print('Data wrangling, the starting time is: %s' % datetime.now())


#    db_cursor = conn.cursor()
#   db_cursor.execute(
#       "SELECT DepartamentName,DepartmentID FROM department")
#   table_rows = db_cursor.fetchall()

#   df_depart = pd.DataFrame(table_rows, columns = ["DepartamentName", "DepartmentID"])

    df_depart = pd.read_csv('dpt.txt', encoding="ISO-8859-1",
                            sep=',')
    df_workers = pd.read_csv('wk.txt', encoding="ISO-8859-1",
                             sep=',')
    display(df_workers)
    display(df_depart)
    display(df_workers.isnull().sum())
    df_workers.filter(df_workers["Budget"] > 10)
    display(df_workers)
    df_aggregated = df_workers.agg({'Budget': ['sum', 'min']})
    display(df_aggregated)

    df_inner = pd.merge(df_depart, df_workers, how='inner', on="DepartmentID")
    display(df_inner)
    df_outer = pd.merge(df_depart, df_workers, how='outer', on="DepartmentID")
    display(df_outer)
    df_left = pd.merge(df_depart, df_workers, how='left', on="DepartmentID")
    display(df_left)
    df_right = pd.merge(df_depart, df_workers, how='right', on="DepartmentID")
    display(df_right)
    df_right.to_csv('df_right_join.txt', sep='\t', index=False)

    grouped = df_workers.groupby('DepartmentID')
    display(grouped.get_group(33))

    one = pd.DataFrame({
         'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Luis'],
         'DepartmentID':['33','26','4','11','34']},
         index=[1,2,3,4,5])
    two = pd.DataFrame({
         'Name': ['Martin', 'Brian', 'Daniel', 'Ron', 'Betty'],
         'DepartmentID':['4','32','6','33','16']},
         index=[1,2,3,4,5])
    display(pd.concat([one,two]))

if __name__ == '__main__':
    scheduler = BackgroundScheduler()
    moveData = move_data
    print(callable(moveData))
    scheduler.add_job(moveData, 'interval', hours=0.005)
    scheduler.start()
    print('Press Ctrl+{0} to exit'.format('Break' if os.name == 'nt' else 'C'))

    try:
        # This is here to simulate application activity
        while True:
            time.sleep(2)
    except (KeyboardInterrupt, SystemExit):
        
        scheduler.shutdown()


True
Press Ctrl+Break to exit
Data wrangling, the starting time is: 2020-09-23 10:07:39.623931


,Name,DepartmentID,Budget
0,Bob,31,27
1,Marcel,33,46
2,Lisa,33,32
3,Thomas,33,25
4,Tim,34,39
5,Josh,34,32
6,Raphael,36,50
7,Dan,36,120


,DepartamentName,DepartmentID
0,Sales,31
1,Marketing,33
2,HumanResources,34
3,DataScience,35


Name            0
DepartmentID    0
Budget          0
dtype: int64

,Name,DepartmentID,Budget
0,Bob,31,27
1,Marcel,33,46
2,Lisa,33,32
3,Thomas,33,25
4,Tim,34,39
5,Josh,34,32
6,Raphael,36,50
7,Dan,36,120


...

,DepartamentName,DepartmentID,Name,Budget
0,Sales,31,Bob,27.0
1,Marketing,33,Marcel,46.0
2,Marketing,33,Lisa,32.0
3,Marketing,33,Thomas,25.0
4,HumanResources,34,Tim,39.0
5,HumanResources,34,Josh,32.0
6,DataScience,35,NaN,NaN
7,NaN,36,Raphael,50.0
8,NaN,36,Dan,120.0


,DepartamentName,DepartmentID,Name,Budget
0,Sales,31,Bob,27.0
1,Marketing,33,Marcel,46.0
2,Marketing,33,Lisa,32.0
3,Marketing,33,Thomas,25.0
4,HumanResources,34,Tim,39.0
5,HumanResources,34,Josh,32.0
6,DataScience,35,NaN,NaN


,DepartamentName,DepartmentID,Name,Budget
0,Sales,31,Bob,27
1,Marketing,33,Marcel,46
2,Marketing,33,Lisa,32
3,Marketing,33,Thomas,25
4,HumanResources,34,Tim,39
5,HumanResources,34,Josh,32
6,NaN,36,Raphael,50
7,NaN,36,Dan,120


,Name,DepartmentID,Budget
1,Marcel,33,46
2,Lisa,33,32
3,Thomas,33,25


,Name,DepartmentID
1,Alex,33
2,Amy,26
3,Allen,4
4,Alice,11
5,Luis,34
1,Martin,4
2,Brian,32
3,Daniel,6
4,Ron,33
5,Betty,16
